<a href="https://colab.research.google.com/github/Favour-Yahdii/Nutrigent/blob/main/Nutrigent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# set up API keys and test fetching data
# store fetched data in a cloud database.
# collect base set of recipes for testing
# spacy to create simple keyword in ingredient to keyword in recipe database link. (prioritize recipes with most overlap in ingredients)
# the generation part (7 days of meals, no ingredient overuse)


In [20]:
from google.colab import userdata

### Set up APIs

In [42]:
# Import relevant functionality
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEndpoint
from typing import List, Dict
from langchain_groq import ChatGroq
from google.colab import userdata
import requests
import json

api_key = userdata.get('spoonacular')
@tool
def get_recipe(
    api_key: str,
    ingredients: List[str],
    number: int,
    ranking: int,
    ignore_pantry: bool) -> Dict[str, any]:
    """
    Get a recipe based on a list of ingredients.
    """
    api_key = userdata.get('spoonacular')
    base_url = "https://api.spoonacular.com/recipes/findByIngredients"
    params = {
        'apiKey': api_key,
        'ingredients': ', '.join(ingredients),
        'number': number,
        'ranking': ranking,
        'ignorePantry': False
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        return json.dumps(data)
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Create the agent
memory = MemorySaver()
groq_key = userdata.get('groq')
llm = ChatGroq(model="llama3-8b-8192", api_key=groq_key)
tools = [get_recipe]
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages":
     [HumanMessage(content="hi im bob! and I would like recipes based on some ingredients I have. My main ingredients are rice and chicken, give me 2 recipes that maximise these ingredients.")]}, config
):
    print(chunk)
    print("----")

# for chunk in agent_executor.stream(
#     {"messages": [HumanMessage(content="what recipes can be made with rice and chicken?", metadata={"apiKey":api_key})]}, config
# ):
#     print(chunk)
#     print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7qdk', 'function': {'arguments': '{"api_key":"your_api_key","ignore_pantry":false,"ingredients":["rice","chicken"],"number":2,"ranking":0,"ignore_pantry":false}', 'name': 'get_recipe'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 1043, 'total_tokens': 1158, 'completion_time': 0.095833333, 'prompt_time': 0.126375399, 'queue_time': 0.02106681499999999, 'total_time': 0.222208732}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-06ce7571-71ea-4d1d-bdf0-571896cb98b1-0', tool_calls=[{'name': 'get_recipe', 'args': {'api_key': 'your_api_key', 'ignore_pantry': False, 'ingredients': ['rice', 'chicken'], 'number': 2, 'ranking': 0}, 'id': 'call_7qdk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1043, 'output_tokens': 115, 'total_tokens': 1158})]}}
----
{'tools

In [ ]:
GET https://api.spoonacular.com/recipes/findByIngredients
